In [1]:
import requests
import re
import numpy as np
import pandas as pd
import scipy as sp
from bs4 import BeautifulSoup
import FinanceDataReader as fdr
from tqdm import tqdm

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 네이버 금융의 코스피 200 페이지 URL
url = "https://finance.naver.com/sise/sise_index.nhn?code=KPI200"

# 웹 페이지 요청
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# 코스피 200 종목이 나오는 링크로 이동
stock_list_url = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0"
response = requests.get(stock_list_url)
soup = BeautifulSoup(response.text, 'html.parser')

# 종목 테이블 가져오기
table = soup.find('table', class_='type_2')
rows = table.find_all('tr')

# 데이터 저장용 리스트
data = []

for row in rows:
    cols = row.find_all('td')
    if len(cols) > 1:  # 빈 행은 제외
        # 필요한 정보만 추출 (종목명, 현재가, 시가총액 등)
        stock_info = {
            "종목명": cols[1].get_text().strip(),
            "현재가": cols[2].get_text().strip(),
            "전일비": cols[3].get_text().strip(),
            "등락률": cols[4].get_text().strip(),
            "거래량": cols[5].get_text().strip(),
            "시가총액": cols[6].get_text().strip()
        }
        data.append(stock_info)

# DataFrame으로 변환
df = pd.DataFrame(data)
print(df)

          종목명        현재가                 전일비     등락률    거래량       시가총액
0        삼성전자     59,200     상승\n\t\t\t\t100  +0.17%    100  3,534,111
1      SK하이닉스    186,300   하락\n\t\t\t\t8,700  -4.46%  5,000  1,356,268
2    LG에너지솔루션    407,500   하락\n\t\t\t\t4,000  -0.97%    500    953,550
3    삼성바이오로직스  1,002,000  하락\n\t\t\t\t29,000  -2.81%  2,500    713,163
4         현대차    215,000   하락\n\t\t\t\t5,500  -2.49%  5,000    450,245
5        셀트리온    182,500   하락\n\t\t\t\t5,200  -2.77%  1,000    396,064
6       삼성전자우     47,950     하락\n\t\t\t\t700  -1.44%    100    394,574
7          기아     91,900   하락\n\t\t\t\t2,200  -2.34%  5,000    367,470
8        KB금융     90,300   하락\n\t\t\t\t3,600  -3.83%  5,000    355,356
9    POSCO홀딩스    335,500   하락\n\t\t\t\t4,500  -1.32%  5,000    277,205
10      NAVER    170,000   하락\n\t\t\t\t4,400  -2.52%    100    273,334
11       신한지주     51,300   하락\n\t\t\t\t3,200  -5.87%  5,000    261,319
12      현대모비스    249,500   상승\n\t\t\t\t1,000  +0.40%  5,000    232,023
13    

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Base URL for KOSPI 200 stock list
base_url = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page="

data = []

# Loop through pages
for page in range(1, 5):
    url = base_url + str(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find stock table
    table = soup.find('table', class_='type_2')
    rows = table.find_all('tr')

    for row in rows:
        cols = row.find_all('td')
        if len(cols) > 1:  # Exclude empty rows
            # 종목 코드 추출
            link = cols[1].find('a')['href']
            code = re.search(r'\d+', link).group() if link else None

            # Remove unnecessary characters and whitespace
            stock_info = {
                "Code": code,
                "Stock Name": cols[1].get_text(strip=True),
                "Current Price": cols[2].get_text(strip=True),
                "Change": cols[3].get_text(strip=True).replace('\n', '').replace('\t', ''),
                "Change Rate": cols[4].get_text(strip=True).replace('\n', '').replace('\t', ''),
                "Volume": cols[5].get_text(strip=True),
                "Market Cap": cols[6].get_text(strip=True)
            }
            data.append(stock_info)

# Convert to DataFrame
df = pd.DataFrame(data)

# Display the result
print(df)

       Code        Stock Name Current Price    Change Change Rate Volume  \
0    005930              삼성전자        59,200     상승100      +0.17%    100   
1    000660            SK하이닉스       186,300   하락8,700      -4.46%  5,000   
2    373220          LG에너지솔루션       407,500   하락4,000      -0.97%    500   
3    207940          삼성바이오로직스     1,002,000  하락29,000      -2.81%  2,500   
4    005380               현대차       215,000   하락5,500      -2.49%  5,000   
..      ...               ...           ...       ...         ...    ...   
195  360200      ACE 미국S&P500        20,085     하락245      -1.21%      0   
196  298020             효성티앤씨       308,500  상승10,500      +3.52%  5,000   
197  010060            OCI홀딩스        68,800   상승1,300      +1.93%  5,000   
198  379810  KODEX 미국나스닥100TR        18,625     하락370      -1.95%      0   
199  294870         HDC현대산업개발        19,820     하락630      -3.08%  5,000   

    Market Cap  
0    3,534,111  
1    1,356,268  
2      953,550  
3      713,163  
4 

In [5]:
price_df = pd.DataFrame()
for stock in tqdm(df['Code']):
    try:
        stock_data = fdr.DataReader(stock, start='2020-1-1')[['Close']]
        stock_data.columns = [stock]
        price_df = pd.concat([price_df, stock_data], axis=1)
    except Exception as e:
        print(f"Failed to retrieve data for stock {stock}: {e}")
        continue

# Display the result

100%|██████████| 200/200 [00:16<00:00, 12.41it/s]


In [10]:
all_data = []
for stock in tqdm(df['Code']):
    try:
        stock_data = fdr.DataReader(
            stock, start='2020-01-01')[['Open', 'High', 'Low', 'Close', 'Volume']]

        # 기본 이동 평균 및 변동성
        stock_data['MA_5'] = stock_data['Close'].rolling(window=5).mean()
        stock_data['MA_20'] = stock_data['Close'].rolling(window=20).mean()
        stock_data['Daily Volatility'] = stock_data['Close'].pct_change()

        # 추가 지표들
        stock_data['Volume Change Rate'] = stock_data['Volume'].pct_change()
        stock_data['High-Low Spread'] = stock_data['High'] - stock_data['Low']
        stock_data['Trading Value'] = stock_data['Close'] * \
            stock_data['Volume']
        stock_data['MACD'] = stock_data['Close'].ewm(span=12, adjust=False).mean(
        ) - stock_data['Close'].ewm(span=26, adjust=False).mean()

        # RSI 계산
        delta = stock_data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        stock_data['RSI'] = 100 - (100 / (1 + gain / loss))

        # Stochastic Oscillator
        stock_data['Stochastic Oscillator'] = ((stock_data['Close'] - stock_data['Low'].rolling(window=14).min()) /
                                               (stock_data['High'].rolling(window=14).max() - stock_data['Low'].rolling(window=14).min())) * 100

        # Momentum
        stock_data['Momentum'] = stock_data['Close'].diff(10)

        # MultiIndex로 컬럼 이름에 종목 코드 추가
        stock_data.columns = pd.MultiIndex.from_product(
            [[stock], stock_data.columns])
        all_data.append(stock_data)
    except Exception as e:
        print(f"Failed to retrieve data for stock {stock}: {e}")
        continue

# 수집된 데이터를 한 번에 병합
price_df = pd.concat(all_data, axis=1)

# 결과 출력
print(price_df.head())

100%|██████████| 200/200 [00:16<00:00, 11.77it/s]


           005930                                                \
             Open   High    Low  Close    Volume     MA_5 MA_20   
Date                                                              
2020-01-02  55500  56000  55000  55200  12993228      NaN   NaN   
2020-01-03  56000  56600  54900  55500  15422255      NaN   NaN   
2020-01-06  54900  55600  54600  55500  10278951      NaN   NaN   
2020-01-07  55700  56400  55600  55800  10009778      NaN   NaN   
2020-01-08  56200  57400  55900  56800  23501171  55760.0   NaN   

                                                                ...   294870  \
           Daily Volatility Volume Change Rate High-Low Spread  ...     MA_5   
Date                                                            ...            
2020-01-02              NaN                NaN            1000  ...      NaN   
2020-01-03         0.005435           0.186946            1700  ...      NaN   
2020-01-06         0.000000          -0.333499            1000 